In [1]:
from IPython.display import Image
from pathlib import Path
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
import math
import xgboost as xgb

In [2]:
train_labels = pd.read_csv("KD/train_labels_KD.csv", index_col="seq_id")
train_labels_columns=train_labels.columns
train_labels['season'] = train_labels.index.map(lambda x: x.split("#")[0])


In [3]:
feature_classes=list(train_labels_columns)
feature_classes.remove('reptiles')
feature_classes.remove('caracal')
feature_classes.remove('wildcat')
feature_classes.remove('honeybadger')
feature_classes.remove('vulture')
feature_classes.remove('duiker')
feature_classes.remove('civet')
feature_classes.remove('genet')
feature_classes.remove('rhinoceros')
feature_classes.remove('zorilla')
feature_classes.remove('cattle')
feature_classes.remove('steenbok')
feature_classes.remove('bat')


In [4]:
everything_S10 = pd.read_csv("KD/everything_fullhd_SER_S10_all.csv", index_col="seq_id")
everything_S9 = pd.read_csv("KD/everything_fullhd_SER_S9_all.csv", index_col="seq_id_buf")
everything_S9.index.name='seq_id'

# everything_s10 = pd.read_csv("KD/everything_SER_S10.csv", index_col="seq_id_buf")
# everything_s10.index.name='seq_id'
len(everything_S10), len(everything_S9)

(252011, 367394)

In [5]:
len(set(everything_S10.index)), len(set(everything_S9.index)), 

(252011, 367394)

In [6]:
# everything_S9 = pd.read_csv("KD/everything_fullhd_SER_S9_all.csv", index_col="seq_id")
# everything_S9 = pd.read_csv("everything_merge.csv", index_col="seq_id")
# everything_S9 = pd.concat([pd.read_csv("KD/everything_fullhd_SER_S9.csv", index_col="seq_id"),  pd.read_csv("everything_merge.csv", index_col="seq_id")])

In [7]:
# everything_S9_add2 = pd.read_csv("KD/everything_fullhd_SER_S9_add2.csv", index_col="seq_id_buf")
# everything_S9_add3 = pd.read_csv("KD/everything_fullhd_SER_S9_add3.csv", index_col="seq_id_buf")
# everything_S9_add4 = pd.read_csv("KD/everything_fullhd_SER_S9_add4.csv", index_col="seq_id_buf")
# 

In [8]:
# everything_S9_add2.iloc[-1]

In [9]:
# everything_S9_add3.iloc[94]

In [10]:
# everything_S9_add3.iloc[-1]

In [11]:
# everything_S9_add4.iloc[13956]

In [12]:
# everything_S9_merge = pd.concat([everything_S9_add2, everything_S9_add3.iloc[94:], everything_S9_add4.iloc[13956:]])
# everything_S9_merge.to_csv("KD/everything_fullhd_SER_S9_all.csv")

In [13]:
# everything_S9 = pd.read_csv("KD/everything_fullhd_SER_S9_add2.csv", index_col="seq_id")
# everything_S9_add= pd.read_csv("KD/everything_fullhd_SER_S9_add4.csv", index_col="seq_id_buf")
# everything_S9_add.index.name='seq_id'
# everything_S9_merge = pd.concat([everything_S9, everything_S9_add.iloc[131:]])
# everything_S9_merge.to_csv("KD/everything_fullhd_SER_S9_all.csv")

In [14]:
# everything_S10_add= pd.read_csv("KD/everything_fullhd_SER_S10_add4.csv", index_col="seq_id_buf")
# everything_S10_add.index.name='seq_id'


In [15]:
# everything_S10_merge = pd.concat([everything_s10, everything_S10_add.iloc[13:]])
# everything_S10_merge.to_csv("KD/everything_fullhd_SER_S10_all.csv")

In [16]:
# len(everything_S9_merge)


In [17]:
# everything_s10['season'] = 10
# everything_S9['season'] = 9

In [18]:
# everything_s10_v2['v2']=True
# everything_s10['v2']=False
# everything_s10['v2']=False

In [20]:
# everything = pd.concat([everything_S9, everything_s10, everything_s10_v2])
everything = pd.concat([everything_S9,  everything_S10])


In [21]:
join = everything.join(train_labels, how='left')


In [22]:
len(join)

619405

In [23]:
len(join[join['season']=='SER_S9'])

367394

In [24]:
len(join[join['season']=='SER_S10'])

252011

In [25]:
models = [
            'pred_model_480_360_as512_img',
            'pred_model_480_360_as480_img',
            'pred_model_480_360_v2_as512_img', 
            'pred_model_480_360_v2_as480_img', 
            'pred_model_512_384_v2_as512_img'
         ]
models2 = [
           'pred_mean_back_2models_as512_',
           'pred_mean_back_2models_as480_',
          ]

models3 = [
           'pred_back_as480_',
           'pred_back_as512_',
          ]


In [26]:
interesting_columns = ['count_imgs', 'season']
models1 = []
for m in models:
    models1.append(m + '1_')
    models1.append(m + '2_')
    models1.append(m + '3_')
for m in models1 + models2 + models3 + [""]:
    for c in feature_classes:
        interesting_columns.append(m + c)
        

In [27]:
join=join[interesting_columns]

In [28]:
# for model_id in ['pred_model_480_360_v2_as480_img1_', 'pred_model_480_360_v2_as480_img1_','pred_model_480_360_v2_as480_img1_',
#          'pred_model_480_360_v2_as512_img1_', 'pred_model_480_360_v2_as512_img2_','pred_model_480_360_v2_as512_img3_',
#          'pred_model_480_360_as480_img1_', 'pred_model_480_360_as480_img2_','pred_model_480_360_as480_img3_',
#          'pred_model_480_360_as512_img1_', 'pred_model_480_360_as512_img2_','pred_model_480_360_as512_img3_',
#           'pred_mean_back_2models_as512_','pred_back_as512_']:
#     join[model_id + 'nonempty'] = join[[model_id + c for c in list(train_labels.columns) if c != 'empty']].sum(axis=1)
#     join[model_id + 'maxnonempty'] = join[[model_id + c for c in list(train_labels.columns) if c != 'empty']].max(axis=1)

In [29]:
for c in feature_classes:
    print(c)
    join['median_img1_' + c] = join[[m + '1_' + c for m in models]].median(axis=1)
    join['median_img2_' + c]=join['median_img1_' + c]
    join['median_img3_' + c]=join['median_img1_' + c]
    join.loc[join.count_imgs > 1, 'median_img2_' + c] = join.loc[join.count_imgs > 1, [m + '2_' + c for m in models]].median(axis=1)
    join.loc[join.count_imgs > 2, 'median_img3_' + c] = join.loc[join.count_imgs > 2, [m + '3_' + c for m in models]].median(axis=1)
    join['mean_imgs_' + c] = (join['median_img1_' + c] + join['median_img2_' + c]*0.8 + join['median_img3_' + c]*0.5) /2.3
    
    join['mean_combines_' + c] = join['mean_imgs_' + c]
    join.loc[join.count_imgs > 1, 'mean_combines_' + c] = join.loc[join.count_imgs > 1,[m  + c for m in models2]].median(axis=1)
    
    join['mean_backs_' + c] = join['mean_imgs_' + c]
    join.loc[join.count_imgs > 1, 'mean_backs_' + c] = join.loc[join.count_imgs > 1,[m  + c for m in models3]].median(axis=1)
    

aardvark
aardwolf
baboon
batearedfox
buffalo
bushbuck
cheetah
dikdik
eland
elephant
empty
gazellegrants
gazellethomsons
giraffe
guineafowl
hare
hartebeest
hippopotamus
hyenaspotted
hyenastriped
impala
insectspider
jackal
koribustard
leopard
lionfemale
lionmale
mongoose
monkeyvervet
ostrich
otherbird
porcupine
reedbuck
rodents
secretarybird
serval
topi
warthog
waterbuck
wildebeest
zebra


In [30]:
DATA_PATH = Path("../../datasets/wildlife") 
train_metadata = pd.read_csv(DATA_PATH / "train_metadata.csv", parse_dates= ['datetime'], index_col="seq_id")
train_metadata = train_metadata.groupby('seq_id').first()
train_metadata['month']=train_metadata.datetime.dt.month
train_metadata['hour']=train_metadata.datetime.dt.hour
join = join.join(train_metadata.hour, how='left')
join = join.join(train_metadata.month, how='left')


In [31]:

join.loc[join.count_imgs<2, ['pred_back_as480_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_back_as512_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_mean_back_2models_as480_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_mean_back_2models_as512_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_as480_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_as480_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_as512_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes]]=-0.01

join.loc[join.count_imgs<3, ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes]]=-0.01
join.loc[join.count_imgs<2, ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes]]=-0.01

           



           

In [32]:

test_index = set(join[join.season=='SER_S10'].index[::2])
test_dataset     = join[join.index.isin(test_index)]
val_dataset = join[(join.season=='SER_S10') & (~join.index.isin(test_index))]
train_dataset = join[join.season=='SER_S9']
animal_columns =list(train_labels.columns)
animal_columns.remove('empty')

# test_index = set(join[join.season=='SER_S10'].index[::5])
# test_dataset = join[join.index.isin(test_index)]
# val_index = set(join[(join.season=='SER_S10') & (~join.index.isin(test_index))].index[::4])
# val_dataset = join[join.index.isin(val_index)]
# train_dataset = join[(~join.index.isin(test_index)) & (~join.index.isin(val_index))]



# (699208, 76659, 76660)
len(train_dataset), len(val_dataset), len(test_dataset)

(367394, 126005, 126006)

In [46]:
training_distibution

empty              0.785650
wildebeest         0.066003
zebra              0.057383
gazellethomsons    0.039426
buffalo            0.005555
elephant           0.006478
hartebeest         0.003239
impala             0.003762
gazellegrants      0.004015
giraffe            0.004292
warthog            0.004693
guineafowl         0.001935
otherbird          0.003424
hyenaspotted       0.004554
lionfemale         0.001747
eland              0.001026
hippopotamus       0.002050
reedbuck           0.001445
topi               0.001644
baboon             0.001162
dikdik             0.000944
secretarybird      0.000857
jackal             0.000808
dtype: float64

In [33]:
import numpy as np
lgbt_models = ['empty', 'wildebeest', 'zebra', 'gazellethomsons', 'buffalo',
'elephant', 'hartebeest', 'impala', 'gazellegrants', 'giraffe',
'warthog', 'guineafowl', 'otherbird', 'hyenaspotted', 'lionfemale', 
'eland', 'hippopotamus', 'reedbuck', 'topi', 'baboon', 'dikdik', 
'secretarybird', 'jackal']

# training_distibution = join.loc[train_labels.season.isin(['SER_S9']), lgbt_models].mean(axis=0)
training_distibution = train_dataset[lgbt_models].mean(axis=0)

test_distibution = train_labels[lgbt_models].mean(axis=0)

scale_pos_weight=test_distibution/training_distibution

for key, value in scale_pos_weight.items():
    # do something with value
    scale_pos_weight[key] = np.clip(value,0.6,1.8)
scale_pos_weight

empty              0.967163
wildebeest         1.155221
zebra              0.936143
gazellethomsons    0.953444
buffalo            1.661107
elephant           1.265842
hartebeest         1.800000
impala             1.763873
gazellegrants      1.592588
giraffe            1.443265
warthog            1.201960
guineafowl         1.800000
otherbird          1.248398
hyenaspotted       0.902223
lionfemale         1.354255
eland              1.800000
hippopotamus       0.983885
reedbuck           1.325607
topi               1.011262
baboon             0.992658
dikdik             1.190888
secretarybird      0.810593
jackal             0.657136
dtype: float64

In [34]:
# empty              0.965829
# wildebeest         1.087374
# zebra              0.987797
# gazellethomsons    1.053421
# buffalo            1.400096
# elephant           1.137100
# hartebeest         1.800000
# impala             1.386069
# gazellegrants      1.800000
# giraffe            1.292890
# warthog            1.215478
# guineafowl         1.800000
# otherbird          1.251465
# hyenaspotted       0.974559
# lionfemale         1.284208
# eland              1.800000
# hippopotamus       0.966361
# reedbuck           1.376051
# topi               1.076195
# baboon             0.924404
# dikdik             0.865440
# secretarybird      0.871325
# jackal             0.688275

In [35]:
from sklearn.metrics import log_loss
loss = 0
for c in feature_classes:
    x = test_dataset[['mean_imgs_' + c, c]].values
    if sum(x[:,1]) !=0:
#         print(c)
        l = log_loss(x[:,1], x[:,0])
        loss+=l
        print(c, l)
print(loss/54)

aardvark 0.00011448686451299324
aardwolf 0.00043729597511019916
baboon 0.0008356553283192734
batearedfox 0.0004525687805471574
buffalo 0.004649831755767214
bushbuck 0.00022754214975594976
cheetah 0.0008176263029226388
dikdik 0.0012679597912138277
eland 0.001985909954805522
elephant 0.004542314817608036
empty 0.046468912787906824
gazellegrants 0.00493314393725784
gazellethomsons 0.007162891533011847
giraffe 0.0014877684568873565
guineafowl 0.0016045421577806074
hare 0.0002419734081486537
hartebeest 0.00334506707158637
hippopotamus 0.0006021332251466067
hyenaspotted 0.0016039266866717564
hyenastriped 0.00032041780880513634
impala 0.004109862442070509
insectspider 0.00506193909758066
jackal 0.0007827833027979016
koribustard 0.000676956069150987
leopard 0.00020572789264205278
lionfemale 0.0022604673769773967
lionmale 0.0012948211812802955
mongoose 0.0007801134795469051
monkeyvervet 0.00040644369353369465
ostrich 0.0007105080024520494
otherbird 0.010259745096893182
porcupine 0.0003713270289

In [37]:
import gc
del train_metadata
del everything_S10
del everything_S9
del train_labels
del join
gc.collect()

8

In [38]:


num_leaves={

}

max_depth={


}
lambda_l1={
    'empty': 0.3,
    'zebra': 0.2,
    'buffalo': 0.3,
}
lambda_l2={
    'empty': 0.3,
    'zebra': 0.2,
    'buffalo': 0.3,
}

learning_rate={
    'zebra':0.008,
    'empty': 0.008,
}

feature_fraction={
    'zebra':0.5,
}
bagging_fraction={
    'zebra':0.8,
    'empty':0.8, #?
}
bagging_freq={
    'zebra':5,
    'empty':5, #?
}
feature_fraction={
    'zebra':0.6,
    'empty':0.3,
}
# elephant
min_child_samples={
    'empty':30,
    'zebra':30,
    'buffalo':30,
    'topi':30,
    'wildebeest':30,
    'gazellethomsons':30
}

steps={
    'empty' : int(907*1.1),
    'wildebeest' : int(865*1.1),
    'zebra' : int(1050*1.1),
    'gazellethomsons' : int(1072*1.1),
    'buffalo' : int(589*1.1),
    'elephant' : int(563*1.1),
    'hartebeest' : int(620*1.1),
    'impala' : int(630*1.1),
    'gazellegrants' : int(616*1.1),
    'giraffe' : int(705*1.1),
    'warthog' : int(752*1.1),
    'guineafowl' : int(416*1.1),
    'otherbird' : int(348*1.1),


}
clazz='empty'

columns = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
    ['mean_backs_' + c for c in feature_classes] +\
    ['median_img1_' + c for c in feature_classes] +\
    ['median_img2_' + c for c in feature_classes] +\
    ['median_img3_' + c for c in feature_classes] +\
    ['mean_combines_' + c for c in feature_classes] +\
    ['hour', 'month']


# columns = ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
#     ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
#     ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
#     ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
#     ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
#     ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
#     ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
#     ['mean_backs_' + c for c in feature_classes] +\
#     ['median_img1_' + c for c in feature_classes] +\
#     ['median_img2_' + c for c in feature_classes] +\
#     ['median_img3_' + c for c in feature_classes] +\
#     ['mean_combines_' + c for c in feature_classes] +\
#     ['hour', 'month']

In [ ]:


# X_train = pd.concat([train_dataset[columns], test_dataset[columns]])
# y_train = pd.concat([train_dataset[clazz], test_dataset[clazz]])
X_train = train_dataset[columns]
y_train = train_dataset[clazz]
X_val = val_dataset[columns]
y_val = val_dataset[clazz]

In [ ]:
len(X_train)

In [ ]:
clazz

In [ ]:
y_train.mean()


In [ ]:
y_val.mean()/y_train.mean()


In [ ]:
scale_pos_weight['warthog']

In [ ]:


lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

scale = y_train.mean()
print(clazz, math.log(1/scale))
params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': {'binary_logloss'},
                'num_leaves': num_leaves.get(clazz, 32),
                'max_depth': max_depth.get(clazz, 6),
                'learning_rate': learning_rate.get(clazz, 0.009),
                'feature_fraction':  feature_fraction.get(clazz, 0.6),
                'bagging_fraction': bagging_fraction.get(clazz, 0.8),
                'bagging_freq': bagging_freq.get(clazz, 5),
                'scale_pos_weight':scale_pos_weight.get(clazz, 1),
                'lambda_l1':lambda_l1.get(clazz, 0.2),
                'lambda_l2':lambda_l2.get(clazz, 0.2),
                'min_child_samples':min_child_samples.get(clazz, 25),

}


gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=3000,
                    valid_sets=[lgb_train, lgb_val],
                    early_stopping_rounds=20)
gbm.save_model("lgbs_v9/" + clazz + '.txt', num_iteration=gbm.best_iteration) 

X_test = test_dataset[columns]
y_test = test_dataset[clazz]

print(clazz, log_loss(y_test, gbm.predict(X_test.values)))
# [784]	training's binary_logloss: 0.0256401	valid_1's binary_logloss: 0.0322288
# empty 0.0310637396503383

# Early stopping, best iteration is:
# [907]	training's binary_logloss: 0.0264165	valid_1's binary_logloss: 0.0319882
# empty 0.030919406864945046

In [113]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

feature_imp = pd.DataFrame(sorted(zip(gbm.feature_importance(),X_test.columns)), columns=['Value','Feature'])
feature_imp[-100:]


,Value,Feature
1384,17,time_wildebeest
1385,18,c2_269
1386,18,c2_272
1387,18,c3_340
1388,18,c3_984
1389,18,mean_combines_baboon
1390,18,mean_with_combines_imgs_baboon
1391,18,median_img3_lionmale
1392,18,pred_model_480_360_as480_img3_otherbird
1393,18,pred_model_512_384_v2_as512_img2_hippopotamus


In [49]:
clazzs = ['empty', 'wildebeest', 'zebra', 'gazellethomsons', 'buffalo',
'elephant', 'hartebeest', 'impala', 'gazellegrants', 'giraffe',
'warthog', 'guineafowl', 'otherbird']


        
for clazz in clazzs:
    
#     feature_classes=feature_classes_d.get(clazz, list(train_labels_columns))

    columns = ['pred_model_480_360_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as480_img3_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_480_360_v2_as512_img3_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img1_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img2_' + c for c in feature_classes] +\
    ['pred_model_512_384_v2_as512_img3_' + c for c in feature_classes] +\
    ['mean_backs_' + c for c in feature_classes] +\
    ['median_img1_' + c for c in feature_classes] +\
    ['median_img2_' + c for c in feature_classes] +\
    ['median_img3_' + c for c in feature_classes] +\
    ['mean_combines_' + c for c in feature_classes] +\
    ['hour', 'month']



    X_train = train_dataset[columns]
    y_train = train_dataset[clazz]
    X_val = val_dataset[columns]
    y_val = val_dataset[clazz]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': {'binary_logloss'},
                'num_leaves': num_leaves.get(clazz, 32),
                'max_depth': max_depth.get(clazz, 6),
                'learning_rate': learning_rate.get(clazz, 0.009),
                'feature_fraction':  feature_fraction.get(clazz, 0.6),
                'bagging_fraction': bagging_fraction.get(clazz, 0.8),
                'bagging_freq': bagging_freq.get(clazz, 5),
                'scale_pos_weight':scale_pos_weight.get(clazz, 1),
                'lambda_l1':lambda_l1.get(clazz, 0.2),
                'lambda_l2':lambda_l2.get(clazz, 0.2),
                'min_child_samples':min_child_samples.get(clazz, 25),

    }

    gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=3000,
                        valid_sets=[lgb_train, lgb_val],
                        early_stopping_rounds=20)
    gbm.save_model("lgbs_v11/" + clazz + '.txt', num_iteration=gbm.best_iteration) 

    X_test = test_dataset[columns]
    y_test = test_dataset[clazz]
    
    print(clazz, log_loss(y_test, gbm.predict(X_test.values)))
    gc.collect()
    

MemoryError: Unable to allocate 3.01 GiB for an array with shape (820, 493399) and data type float64

In [48]:
# Early stopping, best iteration is:
# [909]	training's binary_logloss: 0.0253938	valid_1's binary_logloss: 0.0325604
# empty 0.032561336883759374
# Early stopping, best iteration is:
# [845]	training's binary_logloss: 0.00949369	valid_1's binary_logloss: 0.0205298
# wildebeest 0.02134952140600164
# Early stopping, best iteration is:
# [1190]	training's binary_logloss: 0.00587704	valid_1's binary_logloss: 0.0100063
# zebra 0.010410163679976766
# Early stopping, best iteration is:
# [837]	training's binary_logloss: 0.00760207	valid_1's binary_logloss: 0.00604578
# gazellethomsons 0.006370004320359023
# Early stopping, best iteration is:
# [594]	training's binary_logloss: 0.00131993	valid_1's binary_logloss: 0.00478458
# buffalo 0.0045945335607014
# Early stopping, best iteration is:
# [661]	training's binary_logloss: 0.000644675	valid_1's binary_logloss: 0.00358995
# elephant 0.0037806150024401186
# Early stopping, best iteration is:
# [539]	training's binary_logloss: 0.000635327	valid_1's binary_logloss: 0.00310088
# hartebeest 0.003238209560658141
# Early stopping, best iteration is:
# [559]	training's binary_logloss: 0.00120099	valid_1's binary_logloss: 0.0039591
# impala 0.004122439429679086
# Early stopping, best iteration is:
# [515]	training's binary_logloss: 0.00315171	valid_1's binary_logloss: 0.00524783
# gazellegrants 0.004874318450402127
# Early stopping, best iteration is:
# [663]	training's binary_logloss: 0.000231965	valid_1's binary_logloss: 0.00110938
# giraffe 0.001319812619547732
steps={
    'empty' : int(910),
    'wildebeest' : int(850),
    'zebra' : int(1200),
    'gazellethomsons' : int(900),
    'buffalo' : int(600),
    'elephant' : int(660),
    'hartebeest' : int(540),
    'impala' : int(600),
    'gazellegrants' : int(600),
    'giraffe' : int(663),
    'warthog' : int(752*1.1),
    'guineafowl' : int(416*1.1),
    'otherbird' : int(348*1.1),


}